In [2]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/ALLParenting.csv?token=GHSAT0AAAAAAB6VRXEYIMYHG3L6RTLMFMU6ZALO3KA')

In [4]:
df.head()

,type,url,title,author,date,timestamp,score,text,comment_id,parent_id
0,Post,https://www.reddit.com/r/Parenting/comments/11...,My 10yr old purchased a bunch of stuff on Robl...,invisablecat,2023-03-02,1.677742e+09,5,My 10yr old purchased a bunch of stuff on Robl...,NaN,NaN
1,comment,https://www.reddit.com/r/Parenting/comments/11...,NaN,grandtheftbonsai,2023-03-02,1.677742e+09,29,Roblox is straight predatory. Uninstall and mo...,jalhvb9,NaN
2,comment,https://www.reddit.com/r/Parenting/comments/11...,NaN,mejok,2023-03-02,1.677744e+09,18,> a teachable lesson\n\nThen you need to make ...,jaljx0h,NaN
3,comment,https://www.reddit.com/r/Parenting/comments/11...,NaN,mycodenameisflamingo,2023-03-02,1.677743e+09,12,Make her work around the house aka chores etc ...,jalia21,NaN
4,comment,https://www.reddit.com/r/Parenting/comments/11...,NaN,telmar25,2023-03-02,1.677751e+09,8,Would suggest locking down the iPad. That mean...,jalsobm,NaN


In [7]:
df['text']

0       My 10yr old purchased a bunch of stuff on Robl...
1       Roblox is straight predatory. Uninstall and mo...
2       > a teachable lesson\n\nThen you need to make ...
3       Make her work around the house aka chores etc ...
4       Would suggest locking down the iPad. That mean...
                              ...                        
6882    Suggestion for house chores that my daughter (...
6883    Cleaning baseboards and window sills! All you ...
6884    Putting away dishes.  Helping with laundry.  S...
6885    Personally I only pay the kids for things that...
6886    Her laundry, bathroom, dishes, pet care if you...
Name: text, Length: 6887, dtype: object

In [8]:
#df['comment'].str.count('game').sum()

In [9]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [10]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [11]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: year old with a frequency of 728


In [12]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                term   rank
169931     year old  732.0
129291  screen time  363.0
21811         ca nt  307.0
160683  video games  281.0
50407     feel like  187.0
89117     make sure  181.0
138096  sounds like  174.0


In [13]:
df_ques = df['text'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['text'])

0       My 10yr old purchased a bunch of stuff on Robl...
15                              How old is your daughter?
25      Roblox is like heroin for kids\nAnyone else fe...
105     We’ve never had screen time limits here. My ki...
113     Curious what the consensus is here? Limiting t...
                              ...                        
6813    but his usual homework takes 1 hour a day at l...
6833    Really? You can get scratch on pc. What kind o...
6841    Minecraft\n\nMinecraft\n\n\nMinecraft\n\nMinec...
6851                      Is Roblox on those platforms???
6882    Suggestion for house chores that my daughter (...
Name: text, Length: 372, dtype: object


In [14]:
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    my 10yr old purchased a bunch of stuff on robl...
1    roblox is straight predatory uninstall and mov...
2    > a teachable lesson\n\nthen you need to make ...
3    make her work around the house aka chores etc ...
4    would suggest locking down the ipad that means...
Name: text_processed, dtype: object

In [17]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['yr', 'old', 'purchased', 'bunch', 'stuff', 'roblox', 'toca', 'boca', 'punish', 'daughter', 'changed', 'settings', 'ipad', 'allow', 'purchases', 'noticed', 'morning', 'since', 'changed', 'lot', 'restrictions', 'know', 'yet', 'know', 'knows', 'stealing', 'wrong', 'decided', 'remove', 'apps']


In [18]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
